In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from skopt.space import Integer, Real
import pickle
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../dataset/data.csv')
data = df.select_dtypes(include=['float64', 'int64'])
targets = ["Turbidity", "DO", "Chl-a"]
data.columns

Index(['Turbidity', 'DO', 'Chl-a', 'Discharge', 'Height', 'Temperature', 'B1',
       'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12',
       'WVP', 'MNDWI', 'GNDVI', 'SDDI', 'NDTI', 'BR', 'NDWI', 'NDPI', 'NDCI',
       '2BDA_Chl', 'RR'],
      dtype='object')

In [3]:
def xgb(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    model = XGBRegressor(
        random_state=42,
        n_jobs=-1,
        objective='reg:squarederror'
    )

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model.fit(X_train, y_train)

    os.makedirs("../models/xgb/split", exist_ok=True)
    save_path = f"../models/xgboost/split/{target}.pkl"
    with open(save_path, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved to {save_path}")

    y_pred = model.predict(X_test)
    mae = np.around(mean_absolute_error(y_test, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y_test, y_pred), 2)
    r2 = np.around(r2_score(y_test, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y_test), 2)

    print(f"Performance for {target} (Split):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [4]:
for target in targets:
    xgb(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

FileNotFoundError: [Errno 2] No such file or directory: '../models/xgboost/split/Turbidity.pkl'

In [ ]:
def cvxgb(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    model = XGBRegressor(
        random_state=42,
        n_jobs=-1,
        objective='reg:squarederror'
    )

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    y_pred = cross_val_predict(model, X, y, cv=kf, n_jobs=-1)
    model.fit(X, y)
    
    os.makedirs("../models/xgb/cross", exist_ok=True)
    save_path = f"../models/xgboost/cross/{target}.pkl"
    with open(save_path, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved to {save_path}")

    mae = np.around(mean_absolute_error(y, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y, y_pred), 2)
    r2 = np.around(r2_score(y, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y), 2)

    print(f"Performance for {target} (5-Fold CV):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [ ]:
for target in targets:
    cvxgb(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)